In [1]:
import json
import pandas as pd

In [2]:
with open("../source/StockFacts.json", 'r') as infile:
    data = infile.read()
    new_data = data.replace('}{', '},{')
    json_data = json.loads(f'[{new_data}]')

In [3]:
tickers = pd.read_csv("../source/compcom.csv")
tickers_df = tickers.drop(columns="url")

In [4]:
# # TEST RUN for revenue extraction
# for x in range(len(json_data)):
#     try:
#         #json_data[x]['data']['revenue']
#         print(x, json_data[x]['data']['revenue'])
#     except: 
#         print(x, "NaN")

In [5]:
# # TEST RUN for size extraction
# for x in range(len(json_data)):
#     try: 
#         print(x,json_data[x]['data']['size'])
#     except:
#         print(x,"NaN")

In [6]:
revenue = []
for x in range(len(json_data)):
    try:
        #json_data[x]['data']['revenue']
        revenue.append(json_data[x]['data']['revenue'])
        
    except: 
        revenue.append("NaN")

In [7]:
size = []
for x in range(len(json_data)):
    try:
        #json_data[x]['data']['revenue']
        size.append(json_data[x]['data']['size'])
        
    except: 
        size.append("NaN")

In [8]:
revenue_df = pd.DataFrame(revenue, columns = ['revenue'])

In [9]:
size_df = pd.DataFrame(size, columns = ['size'])

In [11]:
combine = [tickers_df, revenue_df, size_df]
size_revenue_df = pd.concat(combine, axis=1)
size_revenue_df.head()

# figured adding ticker column would make it easier to join on SQL?? yes? no?

,Ticker,revenue,size
0,AAPL,over-1b,over-10k
1,ABNB,200m-1b,5k-10k
2,ADBE,1m-10m,over-10k
3,ADI,over-1b,over-10k
4,ADP,over-1b,over-10k


In [13]:
# if need to change "none" and "NaN" to zero value
# size_revenue_df.fillna(0)

In [12]:
size_revenue_df.to_csv("../source/size_revenue_data.csv")